# Tiny A11y Model Interactive Testing
This notebook lets you test `younglim/tiny-a11y-model` on various accessibility prompts interactively.

Requirements: Free GPU runtime in Colab.

In [ ]:
# Install required packages
!pip install --upgrade pip
!pip install transformers datasets peft huggingface_hub accelerate ipywidgets

In [ ]:
# Login to Hugging Face Hub
from huggingface_hub import notebook_login
notebook_login()  # Paste your HF token when prompted

In [ ]:
# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

model_name = 'younglim/tiny-a11y-model'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
# Define a list of prompts
prompts = [
    'Explain why alt text is important for images.',
    'Give an example of a good label for a button.',
    'Explain WCAG 1.4.1 in simple terms.',
    'How can developers make accessible tables?',
    'Explain ARIA roles for screen readers.'
]

In [ ]:
# Interactive dropdown and button
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=prompts,
    description='Prompt:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)
display(dropdown)

button = widgets.Button(description='Generate Answer')
output = widgets.Output()
display(button, output)

def on_button_click(b):
    with output:
        output.clear_output()
        selected_prompt = dropdown.value
        inputs = tokenizer(selected_prompt, return_tensors='pt')
        outputs_ids = model.generate(**inputs, max_new_tokens=150)
        print(tokenizer.decode(outputs_ids[0], skip_special_tokens=True))

button.on_click(on_button_click)

🎉 Use the dropdown to select a prompt and click **Generate Answer** to see your model's response.